In [2]:
import sympy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [30]:
# mu = 3.986e14
mu = 1.0

# Analytical
def function(p_vec):
    p_norm = np.linalg.norm(p_vec)
    return -mu * p_vec / p_norm**3

def first_order_partial_derivatives(p_vec):
    n = len(p_vec)
    p_norm = np.linalg.norm(p_vec)
    return -mu * (np.eye(n) / p_norm**3 - 3 * np.outer(p_vec, p_vec) / p_norm**5)

def second_order_partial_derivatives(p_vec):
    n = len(p_vec)
    p_norm = np.linalg.norm(p_vec)
    return -mu * (-3 / p_norm**5 * (np.outer(np.eye(n), p_vec) + np.kron(np.eye(n), p_vec) + np.kron(p_vec, np.eye(n))) + 15 / p_norm**7 * np.kron(p_vec, np.outer(p_vec, p_vec))) 

In [34]:
# Symbolic
p1, p2, p3 = sp.symbols('p1 p2 p3')
p_vec = sp.Matrix([p1, p2, p3])
p_norm = sp.sqrt(p1**2 + p2**2 + p3**2)
fun = -mu * p_vec / p_norm**3
first_order_der = sp.derive_by_array(fun, (p1, p2, p3))
first_order_der_fun = sp.lambdify((p1, p2, p3), sp.ImmutableDenseNDimArray(first_order_der).tolist(), 'numpy')
second_order_der = sp.derive_by_array(first_order_der, (p1, p2, p3))
second_order_der_fun = sp.lambdify((p1, p2, p3), sp.ImmutableDenseNDimArray(second_order_der).tolist(), 'numpy')

# Example point
std = 1e-1
sample_p_vec = np.random.multivariate_normal(mean=[0, 0, 0], cov=std*np.eye(3)).reshape((3, 1))

# First Order Partial Derivatives Comparison
first_order_partial_derivatives_symbolic = np.array(first_order_der_fun(*sample_p_vec)).reshape((3, 3))
first_order_partial_derivatives_analytical = first_order_partial_derivatives(sample_p_vec)
print(first_order_partial_derivatives_symbolic)
print(first_order_partial_derivatives_analytical)
print(np.allclose(first_order_partial_derivatives_analytical, first_order_partial_derivatives_symbolic, atol=1e-6))

# Second Order Partial Derivatives Comparison
second_order_partial_derivatives_symbolic = np.array(second_order_der_fun(*sample_p_vec)).reshape((3*3, 3))
second_order_partial_derivatives_analytical = second_order_partial_derivatives(sample_p_vec)
print(second_order_partial_derivatives_symbolic.shape)
print(second_order_partial_derivatives_analytical.shape)
print(np.allclose(second_order_partial_derivatives_analytical, second_order_partial_derivatives_symbolic, atol=1e-6))

M = 100
all_close_results = []
for _ in range(M):
    sample_p_vec = np.random.multivariate_normal(mean=[0, 0, 0], cov=std*np.eye(3)).reshape((3, 1))
    first_order_partial_derivatives_symbolic = np.array(first_order_der_fun(*sample_p_vec)).reshape((3, 3))
    first_order_partial_derivatives_analytical = first_order_partial_derivatives(sample_p_vec)
    second_order_partial_derivatives_symbolic = np.array(second_order_der_fun(*sample_p_vec)).reshape((3*3, 3))
    second_order_partial_derivatives_analytical = second_order_partial_derivatives(sample_p_vec)
    all_close = np.allclose(first_order_partial_derivatives_analytical, first_order_partial_derivatives_symbolic, atol=1e-6) and np.allclose(second_order_partial_derivatives_analytical, second_order_partial_derivatives_symbolic, atol=1e-6)
    all_close_results.append(all_close)
print(f"All Cases Passed?: {all(all_close_results)}")

[[  7.81570159 -18.22536669 -13.40125284]
 [-18.22536669  -0.57635493  10.66646782]
 [-13.40125284  10.66646782  -7.23934666]]
[[  7.81570159 -18.22536669 -13.40125284]
 [-18.22536669  -0.57635493  10.66646782]
 [-13.40125284  10.66646782  -7.23934666]]
True
(9, 3)
(9, 3)
True
All Cases Passed?: True


In [83]:
def differential_equation(x_vec):
    p_vec = x_vec[:3]
    v_vec = x_vec[3:]
    return np.concatenate((v_vec, function(p_vec)))

def first_order_partial_derivatives_for_the_differential_equation(x_vec):
    first_order_pder = np.zeros((6, 6))
    p_vec = x_vec[:3]
    first_order_pder[:3, 3:] = np.eye(3)
    first_order_pder[3:, :3] = first_order_partial_derivatives(p_vec)
    return first_order_pder

def second_order_partial_derivatives_for_the_differential_equation(x_vec):
    second_order_pder = np.zeros((6 * 6, 6))
    p_vec = x_vec[:3]
    aux = second_order_partial_derivatives(p_vec)
    for i in range(3):
        second_order_pder[6*i+3:6*i+6, :3] = aux[i*3:i*3+3, :]
    return second_order_pder

def f(dt, x_old):
    k1 = differential_equation(x_old)
    k2 = differential_equation(x_old + dt / 2 * k1)
    k3 = differential_equation(x_old + dt / 2 * k2)
    k4 = differential_equation(x_old + dt * k3)
    return x_old + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

def Df(dt, x_old):
    n = len(x_old)
    
    k1 = differential_equation(x_old)
    k2 = differential_equation(x_old + dt / 2 * k1)
    k3 = differential_equation(x_old + dt / 2 * k2)

    Dk1 = first_order_partial_derivatives_for_the_differential_equation(x_old)
    Dk2 = first_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k1) @ (np.eye(n) + dt / 2 * Dk1)
    Dk3 = first_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k2) @ (np.eye(n) + dt / 2 * Dk2)
    Dk4 = first_order_partial_derivatives_for_the_differential_equation(x_old + dt * k3) @ (np.eye(n) + dt * Dk3)
    return np.eye(n) + dt / 6 * (Dk1 + 2 * Dk2 + 2 * Dk3 + Dk4)

def Hf(dt, x_old):
    n = len(x_old)
    
    k1 = differential_equation(x_old)
    k2 = differential_equation(x_old + dt / 2 * k1)
    k3 = differential_equation(x_old + dt / 2 * k2)

    Dk1 = first_order_partial_derivatives_for_the_differential_equation(x_old)
    Dk2 = first_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k1) @ (np.eye(n) + dt / 2 * Dk1)
    Dk3 = first_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k2) @ (np.eye(n) + dt / 2 * Dk2)

    Hk1 = second_order_partial_derivatives_for_the_differential_equation(x_old)
    Hk2 = second_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k1) @ (np.eye(n) + dt / 2 * Dk1) + np.kron(np.eye(6), first_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k1)) @ (dt / 2 * Hk1)
    Hk3 = second_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k2) @ (np.eye(n) + dt / 2 * Dk2) + np.kron(np.eye(6), first_order_partial_derivatives_for_the_differential_equation(x_old + dt / 2 * k2)) @ (dt / 2 * Hk2)
    Hk4 = second_order_partial_derivatives_for_the_differential_equation(x_old + dt * k3) @ (np.eye(n) + dt * Dk3) + np.kron(np.eye(6), first_order_partial_derivatives_for_the_differential_equation(x_old + dt * k3)) @ (dt * Hk3)
    return dt / 6 * (Hk1 + 2 * Hk2 + 2 * Hk3 + Hk4)

In [84]:
std = 1e0
sample_x_vec = np.random.multivariate_normal(mean=np.zeros(6,), cov=std*np.eye(6)).reshape((6, 1))

dt = 60.0
f(dt, sample_x_vec), Df(dt, sample_x_vec), Hf(dt, sample_x_vec)

(array([[ 5.67366578e+01],
        [ 3.86259984e-01],
        [-2.91179001e+02],
        [ 9.60325561e-01],
        [-2.39769035e-03],
        [-4.87428026e+00]]),
 array([[-3.28578503e+01, -4.85741701e+01, -1.46535012e+02,
          5.99634501e+01, -2.07837316e-02,  4.26905770e-02],
        [-4.85736366e+01, -9.23197600e+01,  8.21120164e+01,
         -2.07837316e-02,  5.99735512e+01, -5.49224756e-02],
        [-1.46538250e+02,  8.21147328e+01,  1.28192656e+02,
          4.26905770e-02, -5.49224756e-02,  6.00629987e+01],
        [-5.64255265e-01, -8.09596540e-01, -2.44224526e+00,
          9.98629263e-01, -7.75385078e-04,  1.59856645e-03],
        [-8.09578755e-01, -1.55529793e+00,  1.36850206e+00,
         -7.75392100e-04,  9.99008664e-01, -2.05744619e-03],
        [-2.44235318e+00,  1.36859261e+00,  2.12005483e+00,
          1.59860906e-03, -2.05748194e-03,  1.00236549e+00]]),
 array([[-1.86799446e+02, -1.15356295e+01, -3.47999733e+01,
         -1.79079165e-03,  4.47614829e-04, -9.34